In [46]:
library("tidyr")
library("dplyr")
library("nhs.predict")
library("ggplot2")
library("ggsci")

# input data

In [47]:
age.start  <- 26
screen     <- 0     # Clinically detected = 0, screen detected = 1 (webのsymptoms -> 0)
size       <- 2    # Tumour size mm
grade      <- 2     # Tumour grade
nodes      <- 9     # Number positive nodes
er         <- 1     # ER+ = 1, ER- = 0
her2       <- 1     # HER2+ = 1, HER2- = 0, missing = 9
ki67       <- 0     # KI67+ = 1, KI67- = 0, missing = 9
generation <- 0     # Chemo generation 0, 2 or 3 only
horm       <- 0     # Hormone therapy Yes = 1, no = 0
traz       <- 0     # Trastuzumab therapy Yes = 1, no = 0
bis        <- 0     # Bisphosphonate therapy Yes = 1, no = 0
radio      <- 0     # Radiotherapy Yes = 1, no = 0

r.enabled  <- 0     # Radiotherapy enabled = 1, disabled = 0

# screenとgradeの調整

In [21]:
##----------------------------------------------------------------
##[WINTON FIX] Fix inputs
screen    <- ifelse(screen == 2, 0.204, screen) #0,1,2(unknown)
grade     <- ifelse(grade == 9, 2.13, grade) # Tumour grade 1,2,3

# time,ageの設定

In [22]:
## ------------------------------------------------------------------------
time      <- c(1:15)
age       <- age.start - 1 + time
##[WINTON FIX] - Input changed to include grade = 9
grade.val <- ifelse(er==1, grade, ifelse(grade==2 || grade == 3, 1, 0)) # Grade variable for ER neg

# baselineの係数と変数

In [23]:
## ------------------------------------------------------------------------
age.mfp.1   <- ifelse(er==1, (age.start/10)^-2-.0287449295, age.start-56.3254902)
age.beta.1  <- ifelse(er==1, 34.53642, 0.0089827)
age.mfp.2   <- ifelse(er==1, (age.start/10)^-2*log(age.start/10)-.0510121013, 0)
age.beta.2  <- ifelse(er==1, -34.20342, 0)
size.mfp    <- ifelse(er==1, log(size/100)+1.545233938, (size/100)^.5-.5090456276)
size.beta   <- ifelse(er==1, 0.7530729, 2.093446)
nodes.mfp   <- ifelse(er==1, log((nodes+1)/10)+1.387566896, log((nodes+1)/10)+1.086916249)
nodes.beta  <- ifelse(er==1, 0.7060723, .6260541)
grade.beta  <- ifelse(er==1, 0.746655, 1.129091)
screen.beta <- ifelse(er==1, -0.22763366, 0)
her2.beta   <- ifelse(her2==1, 0.2413,
                      ifelse(her2==0, -0.0762,0 ))
ki67.beta   <- ifelse(ki67==1 & er==1, 0.14904,
                      ifelse(ki67==0 & er==1, -0.11333,0 )) #[WINTON FIX] - Missing 3 at the end

pi <- age.beta.1*age.mfp.1 + age.beta.2*age.mfp.2 + size.beta*size.mfp +
  nodes.beta*nodes.mfp + grade.beta*grade.val + screen.beta*screen +
  her2.beta + ki67.beta
pi
grade.beta*grade.val

[1] 1.845037

[1] 1.49331

# baselineと調整項

$$ 
\begin{align}
    Other \ mortality \ : \ prognostic \ index \ (mi)  &= 0.0698252 \times \big((\frac{age}{10}\big)^2 - 34.23391957\big) + Radiotherapy \ による調整項\\\\
    Breast \ cancer \  mortality \ : \ prognostic \ index \ (pi)  &= b'_cx_R
\end{align}
$$

In [24]:
## ----baseline_adjust-----------------------------------------------------
r.prop   <- 0.69 # Proportion of population receiving radiotherapy
r.breast <- 0.82 # Relative hazard breast specifi mortality from Darby et al
r.other  <- 1.07 # Relative hazard other mortality from Darby et al

if (r.enabled == 1) {
  r.base.br  <- log(1/((1-r.prop) + r.prop*r.breast))
  r.base.oth <- log(1/((1-r.prop) + r.prop*r.other))
} else {
  r.base.br   <- 0
  r.base.oth  <- 0
}

In [25]:
## ------------------------------------------------------------------------
# Other mortality prognostic index (mi)
mi <- 0.0698252*((age.start/10)^2-34.23391957) + r.base.oth
mi

[1] -1.918372

In [48]:
# Breast cancer mortality prognostic index (pi)
pi <- age.beta.1*age.mfp.1 + age.beta.2*age.mfp.2 + size.beta*size.mfp +
  nodes.beta*nodes.mfp + grade.beta*grade.val + screen.beta*screen +
  her2.beta + ki67.beta
pi

[1] 1.845037

In [49]:
c     <- ifelse(generation == 0, 0, ifelse(generation == 2, -.248, -.446))
h     <- ifelse(horm==1 & er==1, -0.3857, 0)
h10  <- c(rep(h, 10), rep(-.26+h, 5)) #including both ATLAS and aTTom trials
t     <- ifelse(her2==1 & traz==1, -.3567, 0)
b     <- ifelse(bis==1, -0.198, 0) # Only applicable to menopausal women.

if(r.enabled == 1) {
  r.br  <- ifelse(radio==1, log(r.breast), 0)
  r.oth <- ifelse(radio==1, log(r.other), 0)
} else {
  r.br = 0
  r.oth = 0
}

rx <- tibble(s = rep(0, 15),
             c = c,
             h = h,
             t = t,
             b = b,
             hc = h + c,
             ht = h + t,
             hb = h + b,
             ct = c + t, # It is unlikely that hromone therapy would not be offered
             cb = c + b, # in a woman with ER positive disease
             tb = t + b,
             hct = h + c + t,
             hcb = h + c + b,
             htb = h + t + b,
             ctb = c + t + b,
             hctb = h + c + t + b,
             h10 = h10,
             h10c = h10 + c,
             h10t = h10 + t,
             h10b = h10 + b,
             h10ct = h10 + c + t,
             h10cb = h10 + c + b,
             h10tb = h10 + t + b,
             h10ctb = h10 + c + t + b,
             )

In [50]:
rx <- rx + pi

cols <- ncol(rx)

# Other mortalityの累積死亡率

In [51]:
## ------------------------------------------------------------------------
# Generate cumulative baseline other mortality
base.m.cum.oth <- exp(-6.052919 + (1.079863*log(time)) + (.3255321*time^.5))

# Generate cumulative survival non-breast mortality
s.cum.oth <- exp(-exp(mi+r.oth)*base.m.cum.oth)

# Convert cumulative mortality rate into cumulative risk
m.cum.oth <- 1- s.cum.oth

# Annual other mortality rate 年単位の死亡率
m.oth <- m.cum.oth
for (i in 2:15) {
  m.oth[i] <- m.cum.oth[i] - m.cum.oth[i-1]
}
cat("累積 : ", m.cum.oth,  "\n")
cat("年単位 : ", m.oth, "\n")

累積 :  0.0004779531 0.001155765 0.001985081 0.002953698 0.004056486 0.005291253 0.006657338 0.00815498 0.00978499 0.01154857 0.01344717 0.01548246 0.0176562 0.01997028 0.02242662 
年単位 :  0.0004779531 0.0006778114 0.0008293168 0.0009686164 0.001102788 0.001234768 0.001366085 0.001497641 0.001630011 0.001763577 0.001898607 0.002035285 0.002173744 0.002314075 0.002456337 


# baseline breast mortality 累積死亡率

$$ 
\begin{align}
    ER+ : \ H_c(t)  &= exp\big(0.7424402 - \frac{7.527762}{\sqrt{t}} - 1.812513 \times \frac{log(t)}{\sqrt{t}}\big) \\\\
    ER- : \ H_c(t)  &= exp\big(-1.156036 + \frac{0.4707332}{t^2} - \frac{3.51355}{t}\big)
\end{align}
$$

In [53]:
## ------------------------------------------------------------------------
# Generate cumulative baseline breast mortality
if (er==1) {
  base.m.cum.br <- exp(0.7424402 - 7.527762/time^.5 - 1.812513*log(time)/time^.5)
} else { base.m.cum.br <- exp(-1.156036 + 0.4707332/time^2 - 3.51355/time)
}

# Generate annual baseline breast mortality
base.m.br <- base.m.cum.br
for (i in 2:15) {
  base.m.br[i] <- base.m.cum.br[i] - base.m.cum.br[i-1] }

In [54]:
base.m.cum.br

[1] 0.001130244 0.004216050 0.008622645 0.013873440 0.019670087 0.025820727
 [7] 0.032197633 0.038713243 0.045306119 0.051932379 0.058560259 0.065166559
[13] 0.071734243 0.078250787 0.084707014

In [55]:
# Generate the annual breast cancer specific mortality rate
m.br <- base.m.br*exp(rx)

# Calculate the cumulative breast cancer mortality rate
m.cum.br <- apply(m.br, 2, cumsum)

# Calculate the cumulative breast cancer survival
s.cum.br <- exp(- m.cum.br)
m.cum.br <- 1- s.cum.br

m.br <- m.cum.br
for (j in 1:cols) {
  for (i in 2:15) {
    m.br[i,j] <- m.cum.br[i,j] - m.cum.br[i-1,j]
  }
}

# Cumulative all cause mortality conditional on surviving breast and all cause mortality
m.cum.all <- 1 - s.cum.oth*s.cum.br
s.cum.all <- 100-100*m.cum.all

# Annual all cause mortality
m.all <- m.cum.all
for (j in 1:cols) {
  for (i in 2:15) {
    m.all[i,j] <- m.cum.all[i,j] - m.cum.all[i-1,j]
  }
}

s.cum.br

s,c,h,t,b,hc,ht,hb,ct,cb,⋯,ctb,hctb,h10,h10c,h10t,h10b,h10ct,h10cb,h10tb,h10ctb
0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,⋯,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730,0.9928730
0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,⋯,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722,0.9736722
0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,⋯,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951,0.9468951
0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,⋯,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479,0.9159479
0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,⋯,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569,0.8829569
0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,⋯,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495,0.8492495
0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,⋯,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602,0.8156602
0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,⋯,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121,0.7827121
0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,⋯,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277,0.7507277
0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,⋯,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981,0.7198981


# Dataframe化して確認

In [64]:
results_table <- data.frame(s.cum.all)
select_col <- c('s','h','c','t','b','h10','h10t','h10c','ctb')
results_selected <- results_table[,select_col]
results_selected <- cbind(other, results_selected)
num_col <- ncol(results_selected)
results_selected[c(5,10,15),select_col]
rx

,s,h,c,t,b,h10,h10t,h10c,ctb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752
10,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844
15,57.19313,57.19313,57.19313,57.19313,57.19313,59.97452,59.97452,59.97452,57.19313


s,c,h,t,b,hc,ht,hb,ct,cb,⋯,ctb,hctb,h10,h10c,h10t,h10b,h10ct,h10cb,h10tb,h10ctb
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037
1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,⋯,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037,1.845037


,s,h,c,t,b,h10,h10t,h10c,ctb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752,87.93752
10,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844,71.15844
15,57.19313,57.19313,57.19313,57.19313,57.19313,59.97452,59.97452,59.97452,57.19313


In [39]:
## ------------------------------------------------------------------------
# Proportion of all cause mortality that is breast cancer
prop.br      <- m.br/(m.br + m.oth)
pred.m.br    <- prop.br*m.all
pred.cum.br  <- apply(pred.m.br, 2, cumsum)
pred.m.oth   <- m.all - pred.m.br
pred.cum.oth <- apply(pred.m.oth, 2, cumsum)
pred.cum.all <- pred.cum.br + pred.cum.oth

In [40]:
## ------------------------------------------------------------------------
# rx benefits
# version implemented on web has benefit as difference in breast specific mortality
benefits2.2 <- 100*(pred.cum.all[,1] - pred.cum.all)

In [41]:
## -----------------------------------------------------------------------
#
start <- 6
m.oth <- m.oth[start:15]
m.br <- m.br[start:15,]
m.all <- m.all[start:15,]

# Proportion of all cause mortality that is breast cancer
prop.br      <- m.br/(m.br + m.oth)
pred.m.br    <- prop.br*m.all
pred.cum.br  <- apply(pred.m.br, 2, cumsum)
pred.m.oth   <- m.all - pred.m.br
pred.cum.oth <- apply(pred.m.oth, 2, cumsum)
pred.cum.all <- pred.cum.br + pred.cum.oth

In [42]:
## ------------------------------------------------------------------------
# rx benefits
# version implemented on web has benefit as difference in breast specific mortality
benefits2.2.delay <- 100*(pred.cum.all[,1] - pred.cum.all)